In [1]:
!pip install sentence-transformers torch datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.4 MB/s eta 0:00:00a 0:00:01


In [2]:
from datasets import load_dataset
from torch.utils.data import DataLoader
from sentence_transformers import InputExample

# Load your dataset (replace with your dataset path or use dummy data)
data = [
    {"anchor": "Bali", "positive": "John went to Bali for vacation", "negative": "John went to Paris for vacation"},
    {"anchor": "Paris", "positive": "John went to Paris for vacation", "negative": "John went to Bali for vacation"}
]

# Convert to InputExample format
train_examples = [
    InputExample(texts=[item["anchor"], item["positive"], item["negative"]])
    for item in data
]


In [3]:
from sentence_transformers import SentenceTransformer, losses, models
from sentence_transformers import LoggingHandler
import logging

# Logging for progress tracking
logging.basicConfig(
    format='%(asctime)s - %(message)s',
    level=logging.INFO,
    handlers=[LoggingHandler()]
)

# Load pre-trained model
model_name = "sentence-transformers/all-mpnet-base-v2"
model = SentenceTransformer(model_name)

# Define the dataloader
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

# Define the loss function
train_loss = losses.TripletLoss(model)

# Fine-tune the model
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=30,
    warmup_steps=100,
    output_path="output/contrastive-mpnet"
)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [4]:
from sentence_transformers import util

# Load fine-tuned model
fine_tuned_model = SentenceTransformer("output/contrastive-mpnet")

# Test examples
anchor = "John went to Goa for vacation"
negative = "John went to for vacation"
positive = "Goa"

# Compute embeddings
emb_anchor = fine_tuned_model.encode(anchor, convert_to_tensor=True)
emb_positive = fine_tuned_model.encode(positive, convert_to_tensor=True)
emb_negative = fine_tuned_model.encode(negative, convert_to_tensor=True)

# Compute cosine similarity
sim_positive = util.cos_sim(emb_anchor, emb_positive)
sim_negative = util.cos_sim(emb_anchor, emb_negative)

print("Similarity with positive:", sim_positive.item())
print("Similarity with negative:", sim_negative.item())


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Similarity with positive: 0.851870059967041
Similarity with negative: 0.43118396401405334


In [5]:
from sentence_transformers import losses

# Prepare data: Only anchor-positive pairs are needed
data = [
    {"anchor": "Bali", "positive": "John went to Bali for vacation"},
    {"anchor": "Paris", "positive": "John went to Paris for vacation"}
]

# Convert to InputExample format
train_examples = [
    InputExample(texts=[item["anchor"], item["positive"]])
    for item in data
]

# Define the dataloader
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

# Define the InfoNCE (MultipleNegativesRankingLoss) loss function
train_loss = losses.MultipleNegativesRankingLoss(model)

# Fine-tune the model
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=30,
    warmup_steps=100,
    output_path="output/infonce-mpnet"
)

# Load the fine-tuned model
fine_tuned_model = SentenceTransformer("output/infonce-mpnet")

# Test examples
anchor = "John went to Goa for vacation"
negative = "John went to for vacation"
positive = "Goa"

# Compute embeddings
emb_anchor = fine_tuned_model.encode(anchor, convert_to_tensor=True)
emb_positive = fine_tuned_model.encode(positive, convert_to_tensor=True)
emb_negative = fine_tuned_model.encode(negative, convert_to_tensor=True)

# Compute cosine similarity
sim_positive = util.cos_sim(emb_anchor, emb_positive)
sim_negative = util.cos_sim(emb_anchor, emb_negative)

print("Similarity with positive:", sim_positive.item())
print("Similarity with negative:", sim_negative.item())


Step,Training Loss


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Similarity with positive: 0.8516236543655396
Similarity with negative: 0.42966514825820923


In [7]:
anchor = "John went to Bali for vacation"
negative = "John went to Bahrain for vacation"
positive = "Indonesia"

# Compute embeddings
emb_anchor = fine_tuned_model.encode(anchor, convert_to_tensor=True)
emb_positive = fine_tuned_model.encode(positive, convert_to_tensor=True)
emb_negative = fine_tuned_model.encode(negative, convert_to_tensor=True)

# Compute cosine similarity
sim_positive = util.cos_sim(emb_anchor, emb_positive)
sim_negative = util.cos_sim(emb_anchor, emb_negative)

print("Similarity with positive:", sim_positive.item())
print("Similarity with negative:", sim_negative.item())

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Similarity with positive: 0.5959672331809998
Similarity with negative: 0.1831170618534088
